# Поиск аномалий

Методы обнаружения аномалий, как следует из названия, позволяют находить необычные объекты в выборке. Но что такое "необычные" и совпадает ли это определение у разных методов?

Начнём с поиска аномалий в текстах: научимся отличать вопросы о программировании от текстов из 20newsgroups про религию.

Подготовьте данные: в обучающую выборку возьмите 20 тысяч текстов из датасета Stack Overflow, а тестовую выборку сформируйте из 10 тысяч текстов со Stack Overflow и 100 текстов из класса soc.religion.christian датасета 20newsgroups (очень пригодится функция `fetch_20newsgroups(categories=['soc.religion.christian'])`). Тексты про программирование будем считать обычными, а тексты про религию — аномальными.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score
from sklearn.neighbors import NearestNeighbors
categories_normal = ['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware',
                     'comp.sys.mac.hardware', 'comp.windows.x']
data_stack = fetch_20newsgroups(subset='all', categories=categories_normal,
                                remove=('headers', 'footers', 'quotes'))

limit_train = 20000
if len(data_stack.data) > limit_train:
    indices = np.random.choice(len(data_stack.data), limit_train, replace=False)
    train_texts = [data_stack.data[i] for i in indices]
else:
    train_texts = data_stack.data
data_religion = fetch_20newsgroups(subset='all', categories=['soc.religion.christian'],
                                   remove=('headers', 'footers', 'quotes'))
if len(data_stack.data) > limit_train + 10000:
    mask = np.ones(len(data_stack.data), dtype=bool)
    if 'indices' in locals(): mask[indices] = False
    available_indices = np.where(mask)[0]
    test_normal_indices = np.random.choice(available_indices, 10000, replace=False)
    test_normal_texts = [data_stack.data[i] for i in test_normal_indices]
else:
    test_normal_texts = data_stack.data[:10000]

test_anomaly_texts = data_religion.data[:100]

y_test = np.array([0]*len(test_normal_texts) + [1]*len(test_anomaly_texts))
test_texts = test_normal_texts + test_anomaly_texts

print(f"Train size: {len(train_texts)}")
print(f"Test size (Normal): {len(test_normal_texts)}, (Anomaly): {len(test_anomaly_texts)}")

vectorizer = TfidfVectorizer(max_features=10000)
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

**(1 балл)**

Проверьте качество выделения аномалий (precision и recall на тестовой выборке, если считать аномалии положительным классов, а обычные тексты — отрицательным) для IsolationForest. В качестве признаков используйте TF-IDF, где словарь и IDF строятся по обучающей выборке. Не забудьте подобрать гиперпараметры.

In [ ]:
clf = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.01, random_state=42, n_jobs=-1)
clf.fit(X_train)

y_pred_raw = clf.predict(X_test)

y_pred = np.where(y_pred_raw == -1, 1, 0)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

**(5 баллов)**

Скорее всего, качество оказалось не на высоте. Разберитесь, в чём дело:
* посмотрите на тексты, которые выделяются как аномальные, а также на слова, соответствующие их ненулевым признакам
* изучите признаки аномальных текстов
* посмотрите на тексты из обучающей выборки, ближайшие к аномальным; действительно ли они похожи по признакам?

Сделайте выводы и придумайте, как избавиться от этих проблем. Предложите варианты двух типов: (1) в рамках этих же признаков (но которые, возможно, будут считаться по другим наборам данных) и методов и (2) без ограничений на изменения. Реализуйте эти варианты и проверьте их качество.

In [ ]:

anomaly_indices = np.where(y_pred == 1)[0]

for i in anomaly_indices[:5]:
    label = "Anomaly" if y_test[i] == 1 else "Normal (False Positive)"
    print(f"--- {label} ---")
    print(test_texts[i][:200])
    print("\n")

feature_names = vectorizer.get_feature_names_out()
first_anomaly_vec = X_test[anomaly_indices[0]]
non_zero_indices = first_anomaly_vec.nonzero()[1]
top_words = [feature_names[i] for i in non_zero_indices]
print(f"Слова в первом найденном аномальном объекте: {top_words[:10]}")

nbrs = NearestNeighbors(n_neighbors=3, algorithm='brute', metric='cosine').fit(X_train)
distances, indices = nbrs.kneighbors(first_anomaly_vec)

print("\nБлижайшие соседи из обучающей выборки (StackOverflow):")
for i in range(3):
    print(f"Neighbor {i+1}: {train_texts[indices[0][i]][:200]}...")

### Эксперимент только с изменением датасета

In [ ]:

vectorizer_v2 = TfidfVectorizer(
    stop_words='english',
    max_df=0.8,
    min_df=5,
    sublinear_tf=True
)

X_train_v2 = vectorizer_v2.fit_transform(train_texts)
X_test_v2 = vectorizer_v2.transform(test_texts)

clf_v2 = IsolationForest(n_estimators=100, contamination=0.05, random_state=42, n_jobs=-1)
clf_v2.fit(X_train_v2)

scores = clf_v2.decision_function(X_test_v2)

threshold = np.percentile(scores, 1)
y_pred_v2 = np.where(scores < threshold, 1, 0)

precision_v2 = precision_score(y_test, y_pred_v2)
recall_v2 = recall_score(y_test, y_pred_v2)

print(f"V2 Precision: {precision_v2:.4f}")
print(f"V2 Recall: {recall_v2:.4f}")

### Эксперимент с любыми изменениями

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
X_train_emb = model.encode(train_texts, show_progress_bar=True)
X_test_emb = model.encode(test_texts, show_progress_bar=True)

clf_emb = IsolationForest(n_estimators=100, contamination=0.02, random_state=42)
clf_emb.fit(X_train_emb)

y_pred_emb_raw = clf_emb.predict(X_test_emb)
y_pred_emb = np.where(y_pred_emb_raw == -1, 1, 0)

precision_emb = precision_score(y_test, y_pred_emb)
recall_emb = recall_score(y_test, y_pred_emb)

print(f"Embeddings Precision: {precision_emb:.4f}")
print(f"Embeddings Recall: {recall_emb:.4f}")

Подготовьте выборку: удалите столбцы `['id', 'date', 'price', 'zipcode']`, сформируйте обучающую и тестовую выборки по 10 тысяч домов.

Добавьте в тестовую выборку 10 новых объектов, в каждом из которых испорчен ровно один признак — например, это может быть дом из другого полушария, из далёкого прошлого или будущего, с площадью в целый штат или с таким числом этажей, что самолётам неплохо бы его облетать стороной.

Посмотрим на методы обнаружения аномалий на более простых данных — уж на табличном датасете с 19 признаками всё должно работать как надо!

Скачайте данные о стоимости домов: https://www.kaggle.com/harlfoxem/housesalesprediction/data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score
df = pd.read_csv('kc_house_data.csv')
df_clean = df.drop(columns=['id', 'date', 'price', 'zipcode'])
clean_data = df_clean.sample(n=20000, random_state=42)
train_df = clean_data.iloc[:10000]
test_df_clean = clean_data.iloc[10000:]

anomalies_list = []
base_house = test_df_clean.iloc[0].copy()

a1 = base_house.copy(); a1['lat'] = -33.0; anomalies_list.append(a1)
a2 = base_house.copy(); a2['yr_built'] = 1200; anomalies_list.append(a2)
a3 = base_house.copy(); a3['yr_built'] = 2050; anomalies_list.append(a3)
a4 = base_house.copy(); a4['sqft_living'] = 100000; anomalies_list.append(a4)
a5 = base_house.copy(); a5['floors'] = 100; anomalies_list.append(a5)
a6 = base_house.copy(); a6['bathrooms'] = 200; anomalies_list.append(a6)
a7 = base_house.copy(); a7['sqft_living'] = 0; a7['sqft_above'] = 0; anomalies_list.append(a7)
a8 = base_house.copy(); a8['sqft_living15'] = 50000; a8['sqft_lot15'] = 100; anomalies_list.append(a8)
a9 = base_house.copy(); a9['grade'] = 100; anomalies_list.append(a9)
a10 = base_house.copy(); a10['long'] = 0.0; anomalies_list.append(a10)

test_anomalies_df = pd.DataFrame(anomalies_list)
test_df_final = pd.concat([test_df_clean, test_anomalies_df], ignore_index=True)

# Метки
y_test_house = np.array([0]*len(test_df_clean) + [1]*len(test_anomalies_df))

**Задание 9. (2 балла)**

Примените IsolationForest для поиска аномалий в этих данных, запишите их качество (как и раньше, это precision и recall). Проведите исследование:

Нарисуйте распределения всех признаков и обозначьте на этих распределениях объекты, которые признаны аномальными.

In [ ]:

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score

X_train_house = train_df
X_test_house = test_df_final
feature_predictions = np.zeros((len(X_train_house.columns), len(X_test_house)))
for t in range(1, 10): #times to ensemble

  for i, column in enumerate(X_train_house.columns):

    X_train_single = X_train_house[[column]]
    X_test_single = X_test_house[[column]]

    clf = IsolationForest(
        n_estimators=1000,
        contamination=0.00001,
        random_state=666
    )
    clf.fit(X_train_single)

    pred = clf.predict(X_test_single)
    feature_predictions[i, :] = np.where(pred == -1, 1, 0)

y_pred_ensemble = np.max(feature_predictions, axis=0)


precision = precision_score(y_test_house, y_pred_ensemble)
recall = recall_score(y_test_house, y_pred_ensemble)

print(f"Ensemble Precision: {precision:.4f}")
print(f"Ensemble Recall:    {recall:.4f}")

found_true = np.sum((y_pred_ensemble == 1) & (y_test_house == 1))
found_false = np.sum((y_pred_ensemble == 1) & (y_test_house == 0))

print(f"Найдено истинных аномалий: {found_true} из 10")
print(f"Ложных срабатываний:      {found_false}")